<a href="https://colab.research.google.com/github/mamba124/just_for_fun/blob/master/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_personik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!pip install pymorphy2 
import pymorphy2
!pip install fasttext
import fasttext
from sklearn.naive_bayes import CategoricalNB as naive_b
from sklearn.metrics import f1_score
import keras

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [317]:
def get_representation(model, texts, mode = 0): # 0 - fasttext, 1 - word2vec
    x = [] 
    tmp = [] 
    for text in texts: 
        tmp=[]
        for word in text: 
            if word not in words_pool: representation = np.zeros((100,))
            else:
                if mode == 0: representation = model.get_word_vector(word)
                elif mode == 1: representation = model[word]
            tmp.append(representation)      
        x.append(tmp)
    
    return x

def extend(words, length, step):
    batch =[]
    words_len = len(words)
    index = 0
  
    while (index + length <= words_len):
        batch.append(words[index:index+length])
        index += step

    return batch

def form_set(X,Y, length, step):
    x_ext = []
    y_ext = []
    for i in range(len(X)):
        x_batch = extend(X[i],length, step)
        y_batch = [Y[i] for _ in range(len(x_batch))]
        x_ext.append(x_batch)
        y_ext.append(y_batch)

    x_ext = [x for senten in x_ext for x in senten ]
    y_ext = [y for senten in y_ext for y in senten ]
    return x_ext, y_ext

In [ ]:
train = pd.read_csv('train.csv')
train.head()
classes = train['Класс'].unique()
classes = np.append(classes, 'OTHER')
match = dict(zip(classes,range(len(classes))))
train['Класс'] = train['Класс'].replace(match)

In [ ]:
test = pd.read_csv('test.csv')
test.head()
match = dict(zip(classes,range(len(classes))))
test['Класс'] = test['Класс'].replace(match)
test

,Пример текста,Класс
0,Я хочу сходить в отпуск с 1 по 4 декабря,0
1,Мне нужен отпуск с понедельника,0
2,Как мне получить оптуск,0
3,Я буду отдыхать на следующей неделе,0
4,Какая у меня была зарплата за прошлый месяц?,1
5,Сколько я заработал в январе?,1
6,Пришлите мне рассетный лист,1
7,Сколько всего мне зраплатят в апреле?,1
8,Мне нужно взять больничный на пару дней,2
9,Я хочу сегодня сходить к доктору,2


In [315]:
text_pool = list(map(lambda x: x.split(), train['Пример текста'].values))
y_train = keras.utils.to_categorical(train['Класс'].values, 4)


In [288]:
test_pool = list(map(lambda x: x.split(), test['Пример текста'].values))

y_test = keras.utils.to_categorical(test['Класс'].values, 4)


# FastText

In [ ]:
np.savetxt('train.txt',train['Пример текста'].values, fmt = '%s')
modelFAST = fasttext.train_unsupervised('train.txt', model= 'cbow')


In [ ]:
x_train = get_representation(modelFAST, text_pool, 0)
x_test = get_representation(modelFAST, test_pool, 0)

In [ ]:
x_train, y_train = form_set(x_train,y_train, 3,1)

x_test, y_test = form_set(x_test,y_test, 3,1)

In [ ]:
x_train = np.array(x_train).reshape(-1,300)
x_test = np.array(x_test).reshape(-1,300)

In [ ]:
classifier = OneVsRestClassifier(LogisticRegression(max_iter=300)).fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))


In [ ]:
y_pred = classifier.predict(x_test)
for i in range(len(y_pred)):
    if sum(y_pred[i]) == 0: y_pred[i][-1] = 1  
f1 = f1_score
print(f1(y_test,y_pred, average = 'weighted'))

0.0


# Word2Vec

In [ ]:
from gensim.models import word2vec
modelGENSIM = word2vec.Word2Vec(text_pool, size = 100, window = 10, min_count=1, workers=10,iter=10)


In [ ]:
x_train = get_representation(modelGENSIM, text_pool, 1)
x_test = get_representation(modelGENSIM, test_pool, 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
x_train, y_train = form_set(x_train,y_train, 3,1)

x_test, y_test = form_set(x_test,y_test, 3,1)

In [ ]:
x_train = np.array(x_train).reshape(-1,300)
x_test = np.array(x_test).reshape(-1,300)

In [ ]:
classifier = OneVsRestClassifier(RidgeClassifier(max_iter=300,tol = 1e-5, solver = 'saga')).fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))


In [ ]:
y_pred = classifier.predict(x_test)
for i in range(len(y_pred)):
    if sum(y_pred[i]) == 0: y_pred[i][-1] = 1  
f1 = f1_score
print(f1(y_test,y_pred, average = 'weighted'))

0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [ ]:
y_pred

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

# PyMorphy + One Hot

In [ ]:
morph = pymorphy2.MorphAnalyzer()
text_pool = [[morph.parse(word)[0].normal_form  for word in sentence]for sentence in text_pool]
words_pool = [word for sentence in text_pool for word in sentence]
words_pool = list(set(words_pool))

print(text_pool)
print(words_pool)

word2index = dict(zip(words_pool,range(len(words_pool))))
bow = []
for sent in text_pool:
    tmp = np.zeros((len(words_pool)+1,))
    for word in sent:
        tmp[word2index[word]] = 1
    bow.append(tmp)

x_train = np.array(bow)

[['хотеть', 'в', 'отпуск'], ['я', 'бы', 'в', 'отдохнуть'], ['как', 'я', 'взять', 'отпуск'], ['хотеть', 'отгул', 'на', 'следующий', 'неделя'], ['хотеть', 'улететь', 'в', 'турция'], ['нужно', 'два', 'день', 'отпуск', 'на', 'следующий', 'неделя'], ['уйти', 'в', 'отпуск', 'в', 'октябрь'], ['можно', 'я', 'отдохнуть'], ['я', 'нужно', 'взять', 'пара', 'день', 'выходной'], ['я', 'очень', 'хотеть', 'отдохнуть'], ['сколько', 'я', 'заработать'], ['какой', 'у', 'я', 'зарплата', 'за', 'март'], ['сколько', 'я', 'заплатить', 'в', 'это', 'месяц'], ['почему', 'я', 'заплатить', 'так', 'мало'], ['сколько', 'я', 'перевести', 'деньга'], ['когда', 'прийти', 'зарплата'], ['посмотреть', 'расчётный', 'листок'], ['где', 'взять', 'расчётный', 'лист'], ['когда', 'я', 'заплатить'], ['когда', 'заплатить', 'зарплата'], ['дать', 'ли', 'я', 'зарплата'], ['я', 'заболеть'], ['горло', 'болеть'], ['у', 'я', 'температура'], ['я', 'нужно', 'к', 'врач'], ['хотеть', 'взять', 'больничный'], ['можно', 'я', 'не', 'пойти', 'на', 

In [ ]:
test_pool = [[morph.parse(word)[0].normal_form  for word in sentence]for sentence in test_pool]
test_words_pool = [word for sentence in test_pool for word in sentence]
test_words_pool = list(set(test_words_pool))

print(test_pool)
print(test_words_pool)

bow = []
for sent in test_pool:
    tmp = np.zeros((len(words_pool)+1,))
    for word in sent:
        if word not in words_pool: tmp[-1] = 1
        else: tmp[word2index[word]] = 1
    bow.append(tmp)

x_test = np.array(bow)

[['я', 'хотеть', 'сходить', 'в', 'отпуск', 'с', '1', 'по', '4', 'декабрь'], ['я', 'нужный', 'отпуск', 'с', 'понедельник'], ['как', 'я', 'получить', 'оптускнуть'], ['я', 'быть', 'отдыхать', 'на', 'следующий', 'неделя'], ['какой', 'у', 'я', 'быть', 'зарплата', 'за', 'прошлый', 'месяц?'], ['сколько', 'я', 'заработать', 'в', 'январе?'], ['прислать', 'я', 'рассетный', 'лист'], ['сколько', 'всего', 'я', 'зраплатить', 'в', 'апреле?'], ['я', 'нужно', 'взять', 'больничный', 'на', 'пара', 'день'], ['я', 'хотеть', 'сегодня', 'сходить', 'к', 'доктор'], ['не', 'прийти', 'сегодня', 'на', 'работу,', 'я', 'заболеть'], ['бльничный', 'нужный']]
['апреле?', 'взять', '4', 'зарплата', 'прошлый', 'месяц?', 'хотеть', 'неделя', 'зраплатить', 'получить', 'оптускнуть', 'отпуск', 'больничный', 'день', 'какой', 'рассетный', 'я', 'к', 'нужный', 'за', 'лист', 'январе?', 'нужно', 'прийти', 'у', 'заработать', 'прислать', 'как', 'сколько', 'по', '1', 'работу,', 'пара', 'декабрь', 'с', 'понедельник', 'сходить', 'быть',

In [ ]:
classifier = OneVsRestClassifier(RidgeClassifier(max_iter=300,tol = 1e-5, solver = 'svd')).fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))


In [ ]:
y_pred = classifier.predict(x_test)
for i in range(len(y_pred)):
    if sum(y_pred[i]) == 0: y_pred[i][-1] = 1  
f1 = f1_score
print(f1(y_test,y_pred, average = 'weighted'))

0.8677248677248678


# PyMorphy + TFidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
    
    tfidf_vectorizer = TfidfVectorizer(token_pattern='(\S+)',ngram_range= (1,2), max_df = 0.9, min_df = 5/len(X_train))####### YOUR CODE HERE #######
    tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer.transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

In [ ]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}